In [9]:
import torch
torch.__version__
torch.version.cuda

'11.8'

In [11]:
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-11.8"  # Force CUDA path
os.environ["TORCH_CUDA_ARCH_LIST"] = "7.0;7.5;8.0;8.6"  # Common architectures


In [1]:
import time
import argparse
import os
import os.path as osp
import torch
import numpy as np
import torch.distributed as dist

from mmengine import Config
from mmengine.runner import set_random_seed
from mmengine.logging import MMLogger
from mmseg.models import build_segmentor


In [23]:
import warnings
warnings.filterwarnings("ignore")


In [31]:
!wget https://cloud.tsinghua.edu.cn/f/d1766fff8ad74756920b/?dl=1 -O nonempty.pth

--2025-03-16 19:31:07--  https://cloud.tsinghua.edu.cn/f/d1766fff8ad74756920b/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 101.6.15.69, 2402:f000:1:402:101:6:15:69
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|101.6.15.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/d8883859-a9a3-437f-96ce-6ca6e3f29335/state_dict.pth [following]
--2025-03-16 19:31:08--  https://cloud.tsinghua.edu.cn/seafhttp/files/d8883859-a9a3-437f-96ce-6ca6e3f29335/state_dict.pth
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 229088996 (218M) [application/octet-stream]
Saving to: ‘nonempty.pth’

nonempty.pth        100%[===================>] 218.48M  12.3MB/s    in 17s     

2025-03-16 19:31:25 (13.1 MB/s) - ‘nonempty.pth’ saved [229088996/229088996]



In [24]:

class EnvironmentManager:
    """Handles environment setup, configuration, distributed processing, and logging."""

    def __init__(self, local_rank, args):
        """
        Initialize environment settings and logging.

        Args:
            local_rank (int): Local rank for distributed processing
            args (argparse.Namespace): Command line arguments
        """
        self.local_rank = local_rank
        self.args = args
        self.distributed = False
        self.cfg = None
        self.logger = None

        # Setup environment in sequence
        self._setup_environment()
        self._load_config()
        self._setup_distributed()
        self._setup_logging()

    def _setup_environment(self):
        """Set up environment variables and random seeds for reproducibility."""
        set_random_seed(self.args.seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True

    def _load_config(self):
        """Load configuration from file."""
        self.cfg = Config.fromfile(self.args.py_config)
        self.cfg.work_dir = self.args.work_dir

        # Ensure work directory exists
        os.makedirs(self.args.work_dir, exist_ok=True)

    def _setup_distributed(self):
        """Set up distributed data parallel processing if multiple GPUs are available."""
        if self.args.gpus > 1:
            self.distributed = True

            # Get environment variables for distributed setup
            ip = os.environ.get("MASTER_ADDR", "127.0.0.1")
            port = os.environ.get("MASTER_PORT", "20507")
            hosts = int(os.environ.get("WORLD_SIZE", 1))  # number of nodes
            rank = int(os.environ.get("RANK", 0))  # node id
            gpus = torch.cuda.device_count()  # gpus per node

            if self.local_rank == 0:
                print(f"Initializing DDP: tcp://{ip}:{port}")

            # Initialize process group
            dist.init_process_group(
                backend="nccl",
                init_method=f"tcp://{ip}:{port}",
                world_size=hosts * gpus,
                rank=rank * gpus + self.local_rank
            )

            world_size = dist.get_world_size()
            self.cfg.gpu_ids = range(world_size)
            torch.cuda.set_device(self.local_rank)

            # Suppress prints on non-master processes
            if self.local_rank != 0:
                import builtins
                builtins.print = self.pass_print
        else:
            self.distributed = False

    def _setup_logging(self):
        """Set up logging with timestamp."""
        timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
        log_file = osp.join(self.args.work_dir, f'{timestamp}.log')
        self.logger = MMLogger('selfocc', log_file=log_file)
        MMLogger._instance_dict['selfocc'] = self.logger

        if self.local_rank == 0:
            self.logger.info(f"Configuration:\n{self.cfg.pretty_text}")

    def pass_print(self, *args, **kwargs):
        """Empty print function for non-master processes."""
        pass

    def is_master(self):
        """Check if this process is the master process."""
        return self.local_rank == 0

    def get_logger(self):
        """Get the logger instance."""
        return self.logger

    def get_config(self):
        """Get the configuration instance."""
        return self.cfg


class ModelManager:
    """Handles model creation, initialization, and checkpoint loading."""

    def __init__(self, env_manager):
        """
        Initialize the model manager.

        Args:
            env_manager (EnvironmentManager): Environment manager instance
        """
        self.env = env_manager
        self.cfg = env_manager.get_config()
        self.logger = env_manager.get_logger()
        self.distributed = env_manager.distributed
        self.local_rank = env_manager.local_rank

        self.model = None
        self.raw_model = None

    def initialize(self):
        """Initialize the model and load weights if specified."""
        self._build_model()
        self._load_checkpoint()
        return self.model

    def _build_model(self):
        """Build and initialize the segmentation model."""
        # Import model definitions (ensure this is available in your project)
        try:
            import model
        except ImportError:
            self.logger.warning("Could not import model module. Assuming model definitions are registered.")

        # Build model from config
        self.model = build_segmentor(self.cfg.model)
        self.model.init_weights()

        # Log model size
        n_parameters = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        self.logger.info(f'Number of trainable parameters: {n_parameters:,}')

        # Setup distributed training if needed
        if self.distributed:
            self._setup_distributed_model()
        else:
            self.model = self.model.cuda()
            self.raw_model = self.model

        self.logger.info('Model initialization complete')

    def _setup_distributed_model(self):
        """Setup model for distributed training with SyncBN if needed."""
        # Convert to SyncBN if specified
        if self.cfg.get('syncBN', True):
            self.model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(self.model)
            self.logger.info('Converted to SyncBatchNorm')

        # Setup DDP
        find_unused_parameters = self.cfg.get('find_unused_parameters', False)
        ddp_model_module = torch.nn.parallel.DistributedDataParallel
        self.model = ddp_model_module(
            self.model.cuda(),
            device_ids=[torch.cuda.current_device()],
            broadcast_buffers=False,
            find_unused_parameters=find_unused_parameters
        )
        self.raw_model = self.model.module

    def _load_checkpoint(self):
        """Resume from checkpoint or load pre-trained weights."""
        # Check for latest checkpoint
        self.cfg.resume_from = ''
        if osp.exists(osp.join(self.env.args.work_dir, 'latest.pth')):
            self.cfg.resume_from = osp.join(self.env.args.work_dir, 'latest.pth')

        # Override with command line argument if provided
        if self.env.args.resume_from:
            self.cfg.resume_from = self.env.args.resume_from

        self.logger.info(f'Resume from: {self.cfg.resume_from}')
        self.logger.info(f'Work directory: {self.env.args.work_dir}')

        # Load checkpoint if available
        if self.cfg.resume_from and osp.exists(self.cfg.resume_from):
            self._load_from_checkpoint(self.cfg.resume_from)
        # Otherwise load from pretrained weights if specified
        elif hasattr(self.cfg, 'load_from') and self.cfg.load_from:
            self._load_from_pretrained(self.cfg.load_from)

    def _load_from_checkpoint(self, checkpoint_path):
        """Load model from checkpoint."""
        try:
            map_location = 'cpu'
            ckpt = torch.load(checkpoint_path, map_location=map_location)
            self.raw_model.load_state_dict(ckpt.get("state_dict", ckpt), strict=True)
            self.logger.info(f'Successfully resumed from {checkpoint_path}')
        except Exception as e:
            self.logger.error(f'Failed to load checkpoint: {str(e)}')
            raise

    def _load_from_pretrained(self, pretrained_path):
        """Load pretrained weights."""
        try:
            ckpt = torch.load(pretrained_path, map_location='cpu')
            state_dict = ckpt.get('state_dict', ckpt)

            try:
                load_info = self.raw_model.load_state_dict(state_dict, strict=False)
                self.logger.info(f'Loaded pretrained weights: {load_info}')
            except Exception:
                # Try with weight refinement if regular loading fails
                from misc.checkpoint_util import refine_load_from_sd
                refined_state_dict = refine_load_from_sd(state_dict)
                load_info = self.raw_model.load_state_dict(refined_state_dict, strict=False)
                self.logger.info(f'Loaded pretrained weights with refinement: {load_info}')
        except Exception as e:
            self.logger.error(f'Failed to load pretrained weights: {str(e)}')
            raise

    def get_model(self):
        """Get the initialized model."""
        return self.model


class DatasetManager:
    """Handles dataset loading and preparation."""

    def __init__(self, env_manager):
        """
        Initialize the dataset manager.

        Args:
            env_manager (EnvironmentManager): Environment manager instance
        """
        self.env = env_manager
        self.cfg = env_manager.get_config()
        self.logger = env_manager.get_logger()
        self.distributed = env_manager.distributed

        self.train_loader = None
        self.val_loader = None

    def load_datasets(self, val_only=True):
        """
        Load datasets and create data loaders.

        Args:
            val_only (bool): Whether to only load validation data

        Returns:
            tuple: (train_loader, val_loader) - DataLoader instances
        """
        try:
            from dataset import get_dataloader

            self.train_loader, self.val_loader = get_dataloader(
                self.cfg.train_dataset_config,
                self.cfg.val_dataset_config,
                self.cfg.train_loader,
                self.cfg.val_loader,
                dist=self.distributed,
                val_only=val_only
            )

            self.logger.info('Dataset loaded successfully')

            if val_only:
                self.logger.info(f'Validation dataset size: {len(self.val_loader)}')
            else:
                self.logger.info(f'Training dataset size: {len(self.train_loader)}')
                self.logger.info(f'Validation dataset size: {len(self.val_loader)}')

            return self.train_loader, self.val_loader
        except Exception as e:
            self.logger.error(f'Failed to load dataset: {str(e)}')
            raise

    def get_val_loader(self):
        """Get validation data loader."""
        return self.val_loader


class Evaluator:
    """Handles model evaluation and metrics computation."""

    def __init__(self, env_manager, model_manager, dataset_manager):
        """
        Initialize the evaluator.

        Args:
            env_manager (EnvironmentManager): Environment manager instance
            model_manager (ModelManager): Model manager instance
            dataset_manager (DatasetManager): Dataset manager instance
        """
        self.env = env_manager
        self.model_manager = model_manager
        self.dataset_manager = dataset_manager

        self.cfg = env_manager.get_config()
        self.logger = env_manager.get_logger()
        self.local_rank = env_manager.local_rank
        self.args = env_manager.args

        self.model = model_manager.get_model()
        self.val_loader = dataset_manager.get_val_loader()

    def evaluate(self):
        """
        Run model evaluation.

        Returns:
            dict: Dictionary containing evaluation metrics
        """
        print_freq = self.cfg.print_freq

        # Initialize metrics
        miou_metric = self._setup_metrics()

        # Set model to evaluation mode
        self.model.eval()
        os.environ['eval'] = 'true'

        self.logger.info('Starting evaluation...')
        with torch.no_grad():
            for i_iter_val, data in enumerate(self.val_loader):
                # Process batch
                result_dict = self._process_batch(data)

                # Handle occupancy predictions
                if 'final_occ' in result_dict:
                    self._process_occupancy(result_dict, miou_metric, i_iter_val)

                # Log progress
                if i_iter_val % print_freq == 0 and self.env.is_master():
                    self.logger.info(f'[EVAL] Iter {i_iter_val:5d}')

        # Compute and log final metrics
        miou, iou2 = miou_metric._after_epoch()
        self.logger.info(f'Evaluation results - mIoU: {miou:.4f}, iou2: {iou2:.4f}')
        miou_metric.reset()

        return {'miou': miou, 'iou2': iou2}

    def _setup_metrics(self):
        """Setup evaluation metrics."""
        from misc.metric_util import MeanIoU

        class_names = [
            'barrier', 'bicycle', 'bus', 'car', 'construction_vehicle',
            'motorcycle', 'pedestrian', 'traffic_cone', 'trailer', 'truck',
            'driveable_surface', 'other_flat', 'sidewalk', 'terrain', 'manmade',
            'vegetation'
        ]

        miou_metric = MeanIoU(
            list(range(1, 17)),  # Classes
            17,                  # Number of classes
            class_names,         # Class names for logging
            True,                # Use class weights
            17,                  # Ignore index
            filter_minmax=False  # Don't filter min/max values
        )
        miou_metric.reset()
        return miou_metric

    def _process_batch(self, data):
        """
        Process a single batch of data.

        Args:
            data (dict): Batch data dictionary

        Returns:
            dict: Result dictionary from model inference
        """
        # Move tensors to GPU
        for k in list(data.keys()):
            if isinstance(data[k], torch.Tensor):
                data[k] = data[k].cuda()

        # Extract images and forward pass
        input_imgs = data.pop('img')
        return self.model(imgs=input_imgs, metas=data)

    def _process_occupancy(self, result_dict, miou_metric, iter_idx):
        """
        Process occupancy predictions and update metrics.

        Args:
            result_dict (dict): Model output dictionary
            miou_metric (MeanIoU): Metric instance
            iter_idx (int): Iteration index for visualization
        """
        for idx, pred in enumerate(result_dict['final_occ']):
            pred_occ = pred
            gt_occ = result_dict['sampled_label'][idx]
            occ_mask = result_dict['occ_mask'][idx].flatten()

            # Visualize occupancy if requested
            if self.args.vis_occ:
                self._visualize_occupancy(pred_occ, gt_occ, iter_idx)

            # Update metrics
            miou_metric._after_step(pred_occ, gt_occ, occ_mask)

    def _visualize_occupancy(self, pred_occ, gt_occ, iter_idx):
        """
        Visualize occupancy predictions.

        Args:
            pred_occ (torch.Tensor): Predicted occupancy
            gt_occ (torch.Tensor): Ground truth occupancy
            iter_idx (int): Iteration index for naming
        """
        try:
            from vis import save_occ

            vis_dir = os.path.join(self.args.work_dir, 'vis')
            os.makedirs(vis_dir, exist_ok=True)

            # Save prediction visualization
            save_occ(
                vis_dir,
                pred_occ.reshape(1, 200, 200, 16),
                f'val_{iter_idx}_pred',
                True, 0
            )

            # Save ground truth visualization
            save_occ(
                vis_dir,
                gt_occ.reshape(1, 200, 200, 16),
                f'val_{iter_idx}_gt',
                True, 0
            )
        except Exception as e:
            self.logger.warning(f'Visualization failed: {str(e)}')



In [25]:
class EnvironmentManager:
    """Handles environment setup, configuration, distributed processing, and logging."""

    def __init__(self, local_rank, args):
        """
        Initialize environment settings and logging.

        Args:
            local_rank (int): Local rank for distributed processing
            args (argparse.Namespace): Command line arguments
        """
        self.local_rank = local_rank
        self.args = args
        self.distributed = False
        self.cfg = None
        self.logger = None

        # Setup environment in sequence
        self._setup_environment()
        self._load_config()
        self._setup_distributed()
        self._setup_logging()

    def _setup_environment(self):
        """Set up environment variables and random seeds for reproducibility."""
        set_random_seed(self.args.seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True

    def _load_config(self):
        """Load configuration from file."""
        self.cfg = Config.fromfile(self.args.py_config)
        self.cfg.work_dir = self.args.work_dir

        # Ensure work directory exists
        os.makedirs(self.args.work_dir, exist_ok=True)

    def _setup_distributed(self):
        """Set up distributed data parallel processing if multiple GPUs are available."""
        if self.args.gpus > 1:
            self.distributed = True

            # Get environment variables for distributed setup
            ip = os.environ.get("MASTER_ADDR", "127.0.0.1")
            port = os.environ.get("MASTER_PORT", "20507")
            hosts = int(os.environ.get("WORLD_SIZE", 1))  # number of nodes
            rank = int(os.environ.get("RANK", 0))  # node id
            gpus = torch.cuda.device_count()  # gpus per node

            if self.local_rank == 0:
                print(f"Initializing DDP: tcp://{ip}:{port}")

            # Initialize process group
            dist.init_process_group(
                backend="nccl",
                init_method=f"tcp://{ip}:{port}",
                world_size=hosts * gpus,
                rank=rank * gpus + self.local_rank
            )

            world_size = dist.get_world_size()
            self.cfg.gpu_ids = range(world_size)
            torch.cuda.set_device(self.local_rank)

            # Suppress prints on non-master processes
            if self.local_rank != 0:
                import builtins
                builtins.print = self.pass_print
        else:
            self.distributed = False

    def _setup_logging(self):
        """Set up logging with timestamp."""
        timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
        log_file = osp.join(self.args.work_dir, f'{timestamp}.log')
        self.logger = MMLogger('selfocc', log_file=log_file)
        MMLogger._instance_dict['selfocc'] = self.logger

        if self.local_rank == 0:
            self.logger.info(f"Configuration:\n{self.cfg.pretty_text}")

    def pass_print(self, *args, **kwargs):
        """Empty print function for non-master processes."""
        pass

    def is_master(self):
        """Check if this process is the master process."""
        return self.local_rank == 0

    def get_logger(self):
        """Get the logger instance."""
        return self.logger

    def get_config(self):
        """Get the configuration instance."""
        return self.cfg



In [26]:

class ModelManager:
    """Handles model creation, initialization, and checkpoint loading."""

    def __init__(self, env_manager):
        """
        Initialize the model manager.

        Args:
            env_manager (EnvironmentManager): Environment manager instance
        """
        self.env = env_manager
        self.cfg = env_manager.get_config()
        self.logger = env_manager.get_logger()
        self.distributed = env_manager.distributed
        self.local_rank = env_manager.local_rank

        self.model = None
        self.raw_model = None

    def initialize(self):
        """Initialize the model and load weights if specified."""
        self._build_model()
        self._load_checkpoint()
        return self.model

    def _build_model(self):
        """Build and initialize the segmentation model."""
        # Import model definitions (ensure this is available in your project)
        try:
            import model
        except ImportError:
            self.logger.warning("Could not import model module. Assuming model definitions are registered.")

        # Build model from config
        self.model = build_segmentor(self.cfg.model)
        self.model.init_weights()

        # Log model size
        n_parameters = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        self.logger.info(f'Number of trainable parameters: {n_parameters:,}')

        # Setup distributed training if needed
        if self.distributed:
            self._setup_distributed_model()
        else:
            self.model = self.model.cuda()
            self.raw_model = self.model

        self.logger.info('Model initialization complete')

    def _setup_distributed_model(self):
        """Setup model for distributed training with SyncBN if needed."""
        # Convert to SyncBN if specified
        if self.cfg.get('syncBN', True):
            self.model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(self.model)
            self.logger.info('Converted to SyncBatchNorm')

        # Setup DDP
        find_unused_parameters = self.cfg.get('find_unused_parameters', False)
        ddp_model_module = torch.nn.parallel.DistributedDataParallel
        self.model = ddp_model_module(
            self.model.cuda(),
            device_ids=[torch.cuda.current_device()],
            broadcast_buffers=False,
            find_unused_parameters=find_unused_parameters
        )
        self.raw_model = self.model.module

    def _load_checkpoint(self):
        """Resume from checkpoint or load pre-trained weights."""
        # Check for latest checkpoint
        self.cfg.resume_from = ''
        if osp.exists(osp.join(self.env.args.work_dir, 'latest.pth')):
            self.cfg.resume_from = osp.join(self.env.args.work_dir, 'latest.pth')

        # Override with command line argument if provided
        if self.env.args.resume_from:
            self.cfg.resume_from = self.env.args.resume_from

        self.logger.info(f'Resume from: {self.cfg.resume_from}')
        self.logger.info(f'Work directory: {self.env.args.work_dir}')

        # Load checkpoint if available
        if self.cfg.resume_from and osp.exists(self.cfg.resume_from):
            self._load_from_checkpoint(self.cfg.resume_from)
        # Otherwise load from pretrained weights if specified
        elif hasattr(self.cfg, 'load_from') and self.cfg.load_from:
            self._load_from_pretrained(self.cfg.load_from)

    def _load_from_checkpoint(self, checkpoint_path):
        """Load model from checkpoint."""
        try:
            map_location = 'cpu'
            ckpt = torch.load(checkpoint_path, map_location=map_location)
            self.raw_model.load_state_dict(ckpt.get("state_dict", ckpt), strict=True)
            self.logger.info(f'Successfully resumed from {checkpoint_path}')
        except Exception as e:
            self.logger.error(f'Failed to load checkpoint: {str(e)}')
            raise

    def _load_from_pretrained(self, pretrained_path):
        """Load pretrained weights."""
        try:
            ckpt = torch.load(pretrained_path, map_location='cpu')
            state_dict = ckpt.get('state_dict', ckpt)

            try:
                load_info = self.raw_model.load_state_dict(state_dict, strict=False)
                self.logger.info(f'Loaded pretrained weights: {load_info}')
            except Exception:
                # Try with weight refinement if regular loading fails
                from misc.checkpoint_util import refine_load_from_sd
                refined_state_dict = refine_load_from_sd(state_dict)
                load_info = self.raw_model.load_state_dict(refined_state_dict, strict=False)
                self.logger.info(f'Loaded pretrained weights with refinement: {load_info}')
        except Exception as e:
            self.logger.error(f'Failed to load pretrained weights: {str(e)}')
            raise

    def get_model(self):
        """Get the initialized model."""
        return self.model


In [32]:
import argparse
import torch

# Create a dummy args object with the desired attributes.
args = argparse.Namespace(
    py_config='config/nuscenes_gs25600_solid.py',
    work_dir='./out/',
    resume_from='./prob_64_state_dict.pth',
    seed=42,
    gpus=torch.cuda.device_count(),
    vis_occ=False
)

# EnvironmentManager(0, args)

In [33]:
environmentManger = EnvironmentManager(0, args)

03/16 19:32:37 - mmengine - INFO - Configuration:
anno_root = 'data/nuscenes_cam/'
batch_size = 1
data_aug_conf = dict(
    H=900,
    W=1600,
    bot_pct_lim=(
        0.0,
        0.0,
    ),
    final_dim=(
        864,
        1600,
    ),
    rand_flip=True,
    resize_lim=(
        1.0,
        1.0,
    ),
    rot_lim=(
        0.0,
        0.0,
    ))
data_root = 'data/nuscenes/'
drop_out = 0.1
embed_dims = 128
grad_max_norm = 35
img_norm_cfg = dict(
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    std=[
        58.395,
        57.12,
        57.375,
    ],
    to_rgb=True)
include_opa = True
input_shape = (
    1600,
    864,
)
load_from = 'ckpts/r101_dcn_fcos3d_pretrain.pth'
loss = dict(
    loss_cfgs=[
        dict(
            balance_cls_weight=True,
            empty_label=17,
            lovasz_ignore=17,
            manual_class_weight=[
                1.01552756,
                1.06897009,
                1.30013094,
                1.07253735,
 

In [34]:
modelManager = ModelManager(environmentManger)

In [35]:
modelManager.initialize()

03/16 19:32:49 - mmengine - WARNING - Could not import model module. Assuming model definitions are registered.


KeyError: 'GaussianOccEncoder is not in the mmseg::model registry. Please check whether the value of `GaussianOccEncoder` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'

In [ ]:
def main(local_rank, args):
    """
    Main entry point for model evaluation.

    Args:
        local_rank (int): Local rank for distributed processing
        args (argparse.Namespace): Command line arguments
    """
    # Initialize environment, model, and dataset managers
    env_manager = EnvironmentManager(local_rank, args)
    model_manager = ModelManager(env_manager)
    dataset_manager = DatasetManager(env_manager)

    # Initialize components
    model_manager.initialize()
    dataset_manager.load_datasets(val_only=True)

    # Evaluate the model
    evaluator = Evaluator(env_manager, model_manager, dataset_manager)
    results = evaluator.evaluate()

    # Log final results
    env_manager.get_logger().info(f"Evaluation complete. Final results: {results}")

    return results

In [42]:
!pip install jaxtyping

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.7 MB/s eta 0:00:00


In [3]:
from model.segmentor import *
from model.backbone import *
from model.neck import *

#
#

In [4]:
from model.lifter import *

In [5]:
from model.encoder import *

[1/39] [GCC][c++/pch]/usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch
FAILED: /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch 
g++ -MMD -MT /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch -MF /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch.d -I "/usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include" -I "/usr/local/lib/python3.11/dist-packages/pybind11/include" -I "/usr/include/python3.11" -I "/usr/local/cuda/include" -I "/usr/local/lib/python3.11/dist-packages/include" -O3 -std=c++14 -fPIC -x c++-header -DTV_CUDA -c /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h -o /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch
/usr/local/lib/python3.11/dist-pack

CalledProcessError: Command '['ninja']' returned non-zero exit status 1.

In [2]:
from model.head import *

farthest_point_sampling import error.
[1/39] [GCC][c++/pch]/usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch
FAILED: /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch 
g++ -MMD -MT /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch -MF /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.gch.d -I "/usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include" -I "/usr/local/lib/python3.11/dist-packages/pybind11/include" -I "/usr/include/python3.11" -I "/usr/local/cuda/include" -I "/usr/local/lib/python3.11/dist-packages/include" -O3 -std=c++14 -fPIC -x c++-header -DTV_CUDA -c /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h -o /usr/local/lib/python3.11/dist-packages/cumm/build/core_cc/include/tensorview_bind/TensorViewBind.h.g

CalledProcessError: Command '['ninja']' returned non-zero exit status 1.

In [53]:
!apt-get install -y cuda

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cpp-12 cuda-12-8 cuda-cccl-12-8 cuda-command-line-tools-12-8 cuda-compiler-12-8 cuda-crt-12-8
  cuda-cudart-12-8 cuda-cudart-dev-12-8 cuda-cuobjdump-12-8 cuda-cupti-12-8 cuda-cupti-dev-12-8
  cuda-cuxxfilt-12-8 cuda-demo-suite-12-8 cuda-documentation-12-8 cuda-driver-dev-12-8
  cuda-gdb-12-8 cuda-libraries-12-8 cuda-libraries-dev-12-8 cuda-nsight-12-8
  cuda-nsight-compute-12-8 cuda-nsight-systems-12-8 cuda-nvcc-12-8 cuda-nvdisasm-12-8
  cuda-nvml-dev-12-8 cuda-nvprof-12-8 cuda-nvprune-12-8 cuda-nvrtc-12-8 cuda-nvrtc-dev-12-8
  cuda-nvtx-12-8 cuda-nvvm-12-8 cuda-nvvp-12-8 cuda-opencl-12-8 cuda-opencl-dev-12-8
  cuda-profiler-api-12-8 cuda-runtime-12-8 cuda-sanitizer-12-8 cuda-toolkit-12-8
  cuda-toolkit-12-8-config-common cuda-tools-12-8 cuda-visual-tools-12-8 dctrl-tools default-jre
  default-jre-headless dkms fakeroot fonts-dejavu-cor